In [ ]:
import requests
import json
import pandas as pd
df = pd.read_csv('pre_clean_data.csv')
for index, row in df.iterrows():
    api_address = row['address']
    parameters = {
        "key": "ojSeJRAYm07WLKZhHKRbxXdXTiXmF6o2",
        "location": api_address
    }
    response = requests.get("http://www.mapquestapi.com/geocoding/v1/address", params=parameters)
    data = response.json()['results']
    lat = data[0]['locations'][0]['latLng']['lat']
    lng = data[0]['locations'][0]['latLng']['lng']
    df.at[index, 'lat'] = lat
    df.at[index, 'lng'] = lng

In [ ]:
import folium
map1 = folium.Map(
    location=[49.338315, -119.089960],
    tiles='cartodbpositron',
    zoom_start=12,
)
df.apply(lambda row:folium.CircleMarker(location=[row["lat"], row["lng"]]).add_to(map1), axis=1)
map1

In [ ]:
df.to_csv("../processed/clean_kelowna_housing_data.csv")
df.to_excel('../processed/clean_kelowna_housing_data.xlsx')